# Basic Rag


In [1]:
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from .env file
_ = load_dotenv(find_dotenv())

# Get OpenAI API key
openai_api_key = os.environ["OPENAI_API_KEY"]

print("API Key Loaded:", openai_api_key[:6] + "..." if openai_api_key else "No key found")


API Key Loaded: sk-pro...


In [3]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [12]:
loader = TextLoader("ms_dhoni.txt")
loaded_document = loader.load()

In [13]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks_of_text = text_splitter.split_documents(loaded_document)

In [14]:
len(chunks_of_text)


56

In [15]:
embeddings = OpenAIEmbeddings()

vector_db = FAISS.from_documents(chunks_of_text, embeddings)

In [16]:
vector_db

In [17]:
retriever = vector_db.as_retriever()


In [19]:
response = retriever.invoke("Who is thala in chennai?")
response

[Document(metadata={'source': 'ms_dhoni.txt'}, page_content="Beyond his international career, Dhoni made a massive impact in the Indian Premier League (IPL). As the captain of the \nChennai Super Kings (CSK), he led the team to multiple IPL titles and playoff appearances. His connection with CSK fans runs \ndeep, and he is adored as 'Thala' in Chennai, meaning 'leader' in Tamil. His calmness, humility, and loyalty towards the \nfranchise made him an IPL legend.\n\nOff the field, Dhoni is known for his simplicity and grounded personality. Despite his enormous success, he has remained \nhumble and approachable. He also has a passion for the Indian Army, holding the honorary rank of Lieutenant Colonel in the \nTerritorial Army. His love for bikes and cars is well known, and he has one of the most impressive collections among \ncricketers."),
 Document(metadata={'source': 'ms_dhoni.txt'}, page_content="Beyond his international career, Dhoni made a massive impact in the Indian Premier Leagu

In [20]:
len(response)


4

In [26]:
retriever = vector_db.as_retriever(search_kwargs={"k": 1})
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x122267b50>, search_kwargs={'k': 1})

In [27]:
response = retriever.invoke("what is cks?")
response

[Document(metadata={'source': 'ms_dhoni.txt'}, page_content="Beyond his international career, Dhoni made a massive impact in the Indian Premier League (IPL). As the captain of the \nChennai Super Kings (CSK), he led the team to multiple IPL titles and playoff appearances. His connection with CSK fans runs \ndeep, and he is adored as 'Thala' in Chennai, meaning 'leader' in Tamil. His calmness, humility, and loyalty towards the \nfranchise made him an IPL legend.\n\nOff the field, Dhoni is known for his simplicity and grounded personality. Despite his enormous success, he has remained \nhumble and approachable. He also has a passion for the Indian Army, holding the honorary rank of Lieutenant Colonel in the \nTerritorial Army. His love for bikes and cars is well known, and he has one of the most impressive collections among \ncricketers.")]

# Simple use with LCEL

In [28]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [29]:
template = """Answer the question based only on the following context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [30]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [38]:
response = chain.invoke("Which three ICC trophies did Dhoni win as captain, and why does this achievement set him apart from other cricket leaders?")
response

'Dhoni won the inaugural T20 World Cup in 2007, the 2011 ICC Cricket World Cup, and the 2013 ICC Champions Trophy as captain. This achievement sets him apart from other cricket leaders because he is the only captain to have won all three major ICC trophies.'

In [41]:
response = chain.invoke("what is dhoni salary?")
response

"Based on the context provided, it is not specified what Dhoni's salary is."

In [44]:
type(response)

openai.types.chat.chat_completion.ChatCompletion